## Simulating RFIM

In [ ]:
import numpy as np
DTYPE = np.float32
from numba import njit
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [ ]:
"""
T: Temperature
L: Lattice Size
J: Interaction Strength
h: External Field
h_rand: Random local fields
"""
@njit
def calculate_energy(spins, L, J, h_rand, h):
    energy = 0
    for i in range(L):
        for j in range(L):
            neighbors = spins[(i+1)%L, j] + spins[(i-1)%L, j] + spins[i, (j+1)%L] + spins[i, (j-1)%L]
            energy += -J*spins[i, j]*neighbors
            energy += -(h_rand[i, j])* spins[i, j]
            energy += -(h)* spins[i, j]
    return energy

# Function to calculate the magnetization of the system
@njit
def calculate_magnetization(spins):
    return np.sum(spins)
    
@njit
def monte_carlo_step(spins, L, J, T, h_rand, h):
    for _ in range(L * L):
        i, j = np.random.randint(0, L, size=2)  
        
        neighbors = spins[(i+1)%L, j] + spins[(i-1)%L, j] + spins[i, (j+1)%L] + spins[i, (j-1)%L]
        delta_E = 2*J*spins[i, j]*(neighbors + h_rand[i, j] + h)

        if delta_E < 0 or np.random.rand() < np.exp(-delta_E / T):
            spins[i, j] *= -1 


In [ ]:
L = int(512)
# T = DTYPE(np.linspace(1, 20, 50)) # At even lower temperature, equilibriation time difficult to find because of domain growth
T = DTYPE(2)
J = DTYPE(1)
# hs =  DTYPE(np.linspace(0.1, 5, 25))
h = 0

# Random Fields (normally distributed)
std = 0.001
h_rand = np.random.normal(0, std, size= (L, L))

# MC params
mc_steps = int(1000)
eq_steps = int(mc_steps/2)

# Initial config
spins = np.random.choice([-1, 1], size=(L, L))  # Random initial spins


In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))

ax[0].imshow(spins)
ax[0].set_xticks(np.arange(0, L, L-1));
ax[0].set_yticks(np.arange(0, L, L-1));
ax[0].set_title("Initial spin configuration")

ax[1].imshow(h_rand)
ax[1].set_xticks(np.arange(0, L, L-1));
ax[1].set_yticks(np.arange(0, L, L-1));
ax[1].set_title("randomized fields")

title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}, T={T:.3f}" + "\n" + f"h_rand = N(0, {std:.3f})"
fig.suptitle(title)
fig.tight_layout()

In [ ]:
energies = []
magnetizations = []

for step in tqdm(range(mc_steps)):

    monte_carlo_step(spins, L, J, T, h_rand, h)
    
    if step % 10 == 0:
        energies.append(calculate_energy(spins, L, J, h_rand, h))
        magnetizations.append(np.mean(spins))

        
        fig, ax = plt.subplots()
        plt.imshow(spins)
        ax.set_xticks(np.arange(0, L, L-1));
        ax.set_yticks(np.arange(0, L, L-1));
        title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}, T={T:.3f}" + "\n" + f"h_rand = N(0, {std:.3f})" +"\n" + f"step {step}"
        fig.suptitle(title)
        fig.tight_layout()


In [ ]:

fig, ax = plt.subplots(1, 2, figsize = (10, 5))

ax[0].plot(energies, label="Energy")
ax[0].set_xlabel("Monte Carlo Steps")
ax[0].set_ylabel("Energy")
ax[0].set_title("Energy vs. Monte Carlo Steps")

ax[1].plot(magnetizations, label="Magnetization", color="orange")
ax[1].set_xlabel("Monte Carlo Steps")
ax[1].set_ylabel("Magnetization")
ax[1].set_title("Magnetization vs. Monte Carlo Steps")

title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}, T={T:.3f}" + "\n" + f"h_rand = N(0, {std:.3f})"
fig.suptitle(title)
fig.tight_layout()

## Finding the critical temperature

In [ ]:
L = int(64)
T = DTYPE(np.linspace(1, 20, 50)) # At even lower temperature, equilibriation time difficult to find because of domain growth
J = DTYPE(1)
hs =  DTYPE(np.linspace(0.1, 5, 25))

mc_steps = int(1000)
eq_steps = int(mc_steps/4)


# Random Fields (normally distributed)
std = 0.001
# h_rand = np.random.normal(0, std, size= (L, L))


In [ ]:
for h in hs:
    magnetizations = []
    susceptibilities = []
    specific_heats = []

    for temp in tqdm(T):
        spins = np.random.choice([-1, 1], size = (L, L))
        h_rand = np.random.normal(0, std, size= (L, L))
        
        M_list = [] # Stores magnetisation after every step
        E_list = [] # Stores energy after every step
        
        for step in range(mc_steps):
            # Uncomment to save the evolution of the system
            # fig, ax = plt.subplots(figsize=(6, 6))
            # plt.imshow(spins)
            # ax.set_xticks(np.arange(0, L, L-1));
            # ax.set_yticks(np.arange(0, L, L-1));
            # title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}, T={temp:.3f}" + "\n" + f"step {step}"
            # fig.suptitle(title)
        
            # output_filepath = f"data/L-{L}/J-{J:.3f}/h-{h:.3f}/T-{temp:.3f}/lattice_evolution"
            # output_filename = f"step{step}.png"
            # if not os.path.exists(output_filepath):
            #     os.makedirs(output_filepath)
        
            # file = os.path.join(output_filepath, output_filename)
            # plt.savefig(file, dpi = 100)
            # plt.close()
            
            if step > eq_steps:
                M = np.abs(calculate_magnetization(spins)) / (L * L)
                E = calculate_energy(spins, L, J, h_rand, h)
                M_list.append(M)
                E_list.append(E)                
            
            monte_carlo_step(spins, L, J, temp, h_rand,  h)
                
        # Compute averages
        M_avg = np.mean(M_list)
        M2_avg = np.mean(np.square(M_list))
        E_avg = np.mean(E_list)
        E2_avg = np.mean(np.square(E_list))
            
        magnetizations.append(M_avg)
        susceptibilities.append((M2_avg - M_avg**2)/temp)
        specific_heats.append((E2_avg - E_avg**2)/(temp**2))
    
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    
    # Magnetization
    ax[0].plot(T, magnetizations, 'o-')
    ax[0].set_xlabel("Temperature (T)")
    ax[0].set_ylabel("Magnetization (M)")
    ax[0].set_title("Magnetization vs. Temperature")
    
    # Susceptibility
    ax[1].plot(T, susceptibilities, 'o-')
    ax[1].set_xlabel("Temperature (T)")
    ax[1].set_ylabel("Susceptibility (χ)")
    ax[1].set_title("Susceptibility vs. Temperature")
    
    # Specific Heat
    ax[2].plot(T, specific_heats, 'o-')
    ax[2].set_xlabel("Temperature (T)")
    ax[2].set_ylabel("Specific Heat (C)")
    ax[2].set_title("Specific Heat vs. Temperature")
    
    title = f"L={L}" + "\n" + f"J={J:.3f}, h={h:.3f}, h_rand = N(0, {std:.3f})"
    
    fig.suptitle(title)
        
    plt.tight_layout()
    output_filepath = f"data/2d/L-{L}/J-{J:.3f}/"
    output_filename = f"h-{h:.3f}.png"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    
    file = os.path.join(output_filepath, output_filename)
    plt.savefig(file, dpi = 100)
    plt.close()